In [1]:
import json
import numpy as np
import pandas as pd

In [3]:
# get calinet
with open('../data/ingestion_tmp_data/calinet_input_information.json', 'r') as f:
    data_calinet = json.load(f)
    calinet_df = pd.DataFrame(data_calinet).T

In [4]:
relations_used = []

In [5]:
stems_used = []

In [6]:
correct = []

In [7]:
relations_used = list(calinet_df['relation'].values)

In [8]:
stems_used = list(calinet_df['stem'].values)

In [9]:
correct = list(calinet_df['true'])

In [10]:
paper = ['calinet'] * len(stems_used)

In [11]:
assert(len(relations_used) == len(stems_used) == len(paper) == len(correct))

In [12]:
# get rome
with open('../data/rome_counterfact_original/counterfact.json', 'r') as f:
    data_rome = json.load(f)
    rome_df = pd.DataFrame(data_rome)

In [13]:
# get stems + relations for rome
for k in range(rome_df.shape[0]):
    paper.append('memit')
    stems_used.append(data_rome[k]['requested_rewrite']['prompt'].replace('{}', data_rome[k]['requested_rewrite']['subject']))
    relations_used.append(rome_df['requested_rewrite'][k]['relation_id'])
    correct.append(rome_df['requested_rewrite'][k]['target_true']['str'])    

In [14]:
agg_df = pd.DataFrame({'relation': relations_used, 'stem': stems_used, 'paper': paper, 'correct': correct})

In [15]:
relation_ids = list(set(agg_df['relation']))

In [16]:
# lookup wikidata vals
wikidata_df = pd.read_csv('../data/wikidata/wikidata-property-list.csv')

In [17]:
relation_titles = []
for rel_id in relation_ids:
    relation_titles.append(list(wikidata_df[wikidata_df['ID'] == int(rel_id[1:])]['Title'])[0])

In [18]:
relation_lookup = dict(zip(relation_ids, relation_titles))
relation_titles_full = []

for i in range(agg_df.shape[0]):
    relation_titles_full.append(relation_lookup[agg_df['relation'][i]])

assert(len(relation_titles_full) == agg_df.shape[0])

In [19]:
agg_df['relation_title'] = relation_titles_full

In [20]:
agg_df.head()

,relation,stem,paper,correct,relation_title
0,P47,Norfolk borders with,calinet,Suffolk,shares border with
1,P47,Jordan shares a common border with,calinet,Israel,shares border with
2,P47,Kenya shares border with,calinet,Ethiopia,shares border with
3,P47,Egypt shares its border with,calinet,Israel,shares border with
4,P47,Tanzania borders with,calinet,Uganda,shares border with


In [21]:
agg_df.groupby(["relation", "relation_title"]).size()

relation  relation_title                             
P101      field of work                                  1045
P103      native language                                 919
P106      occupation                                     1308
P108      employer                                        350
P127      owned by                                        433
P1303     instrument                                      513
P131      is in the administrative territorial entity     714
P136      genre                                          1313
P1376     capital of                                      500
P138      named after                                     779
P140      religion                                        430
P1412     languages spoken                                924
P159      headquarters location                          1252
P17       country                                         875
P176      manufacturer                                   1391
P178      develo

In [28]:
agg_df.groupby(["paper", "relation_title"]).size()

paper    relation_title                             
calinet  capital                                        500
         capital of                                     500
         country of citizenship                         495
         country of origin                              500
         developer                                      462
         diplomatic relation                            494
         field of work                                  500
         formation location                             500
         genre                                          466
         headquarters location                          496
         language                                       491
         manufacturer                                   480
         member of                                      443
         named after                                    500
         occupation                                     487
         official language                     

In [29]:
agg_df[agg_df['paper'] == 'memit'].groupby(["relation_title"]).size()

relation_title
capital                                        139
continent                                      959
country                                        875
country of citizenship                         958
country of origin                              904
developer                                      579
employer                                       350
field of work                                  545
formation location                             774
genre                                          847
headquarters location                          756
instrument                                     513
is in the administrative territorial entity    714
language                                       216
languages spoken                               924
located in                                     613
manufacturer                                   911
member of                                      163
named after                                    279
native language 

In [30]:
agg_df[agg_df['paper'] == 'calinet'].groupby(["relation_title"]).size()

relation_title
capital                    500
capital of                 500
country of citizenship     495
country of origin          500
developer                  462
diplomatic relation        494
field of work              500
formation location         500
genre                      466
headquarters location      496
language                   491
manufacturer               480
member of                  443
named after                500
occupation                 487
official language          462
original language          493
original network           500
place of birth             497
place of death             499
position held              486
position played on team    245
record label               500
shares border with         500
work location              464
dtype: int64

In [35]:
agg_df[agg_df['relation_title'] == 'country of origin'].groupby(["paper"]).size()

paper
calinet    500
memit      904
dtype: int64

In [37]:
religion = agg_df[agg_df['relation_title'] == 'religion']
religion.to_csv('../data/ingestion_tmp_data/religion_prompts.csv', index=False)